# Creating material dictionary
# [TODO] - extract out the page logics while deleting other create dictionary files

- same them into csv files.

In [3]:
import numpy as np
import pandas as pd
import re
import math
import unicodedata

In [4]:
# file paths
size_file = "raw_data/サイズ表記.xlsx"
category_file = "raw_data/商品分類表_180410.xlsx"

# # import excel files
# # for size data
size_df = pd.read_excel( size_file, sheet_name='Q_最終メモ', header=0, index_col=0 )
size_df.columns = ['size','quality','brand']
size_df.index.name = 'product_id'

# for category data
category_df = pd.read_excel(category_file,  sheet_name='Sheet1', index_col=0 )
category_df.columns = ['product_name',
                       'material_type_id',
                       'material_type',
                       'manufacture_country_id',
                       'manufacture_country',
                       'category_id',
                       'category']
category_df.index.name = 'product_id'

material_df = size_df
material_df['product_name'] = category_df['product_name']

material_df

,size,quality,brand,product_name
product_id,,,,
000001,NaN,NaN,NaN,志成貿易 輸入代行手数料
011007BE,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007GN,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007LB,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007LG,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007NT,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007NV,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007OR,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０
011007PK,NaN,NaN,NaN,ＢＣ イタワプレーン ダブル ２２０ｘ２７０


In [40]:
query_keywords = [
    'ＮＹＬＯＮ',
    'ＰＡＰＥＲ',
    'ＬＩＮＥＮ',
    'アクリル',
    'ＰＥＷＴＥＲ',
    'ＰＯＬＹＥＴＨＹＬＥＮＥ',
    'レーヨン',
    'ＦＡＫＥＬＥＡＴＨＥＲ',
    'ガルバルーフ',
    'WOOD',
    '毛',
    'ニット',
    'ＳＥＡＧＲＡＳＳ',
    'WOOL',
    'ｅｎａｍｅｌｗａｒｅ',
    'ＰＡＲＡＦＦＩＮ',
    'ＳＩＬＫ',
    'ツイストウール',
    'ＳＩＬＶＥＲ９２５',
    'ＢＲＡＳＳ',
    'ＰＥＡＲＬ',
    'ブルーミング',
    'SILK',
    'ガラス',
    'PINE',
    'ＷＯＯＬ',
    'ＧＯＬＤ ＰＬＡＴＥＤ',
    'ポリエステル',
    '麻',
    'ＣＯＴＴＯＮ',
    'フラッフィーウール',
    'ＡＣＲＹＬＩＣ',
    '綿',
    'ＰＯＬＹＥＳＴＥＲ',
    'ＩＲＯＮ',
    'ＰＡＬＭ ＷＡＸ',
    '合成皮革',
    'FUR',
    'ナイロン',
    'リサイクルレザー',
    'ＧＬＡＳＳ',
    'ＧＡＬＶＡＮＩＺＥＤ',
    'ソリッドファブリック',
    'ブリキ',
    'ＡＬＬＯＹ',
    'ＭＡＨＯＧＡＮＹ',
    'ＲＡＦＦＩＡ',
    'ＣＯＷ ＬＥＡＴＨＥＲ',
    'ＰＶＣ',
    'レジン',
    'ＴＩＮ',
    'ＷＯＯＤ',
    'ＳＴＲＡＷ',
    'ＳＴＯＮＥ',
    'ＰＬＡＳＴＩＣ',
    'ＰＡＲＡＦＩＮ',
    'ＪＵＴＥ',
    'ＦＬＡＸ',
    'ＰＯＬＹ',
    'ＡＢＡＣＡ',
    'ＢＥＥＳＷＡＸ',
    'ＥＮＡＭＥＬ',
    'ＰＵＬＰ',
    'ＳＨＥＥＰ ＬＥＡＴＨＥＲ',
    'ＭＩＲＲＯＲ',
    'ＳＴＡＩＮＬＥＳＳ',
    'ＲＥＳＩＮ',
    'ＡＲＵＲＯＧ',
    'ＣＥＲＡＭＩＣ',
    'ＡＬＵＭＩＮＩＵＭ',
    'ＣＥＤＡＲ',
    'ＴＥＲＲＡＣＯＴＴＡ',
    'ＭＩＮＫ ＦＵＲ',
    'ＳＯＲＧＨＵＭ',
    'ＡＣＡＣＩＡ',
    'ＪＡＣＫＦＵＲＵＩＴ ＬＥＡＦ',
    'ＮＡＮＧＫＡ',
    'ＣＥＬＬＵＬＯＩＤ',
    'ＬＩＬＹ',
    'ＰＯＲＣＥＬＡＩＮ',
    '花梨',
    'ＳＷＡＲＯＶＳＫＩ',
    'ＳＰＡＮＧＬＥ',
    'ＲＡＢＢＩＴ ＦＵＲ',
    'ＲＡＹＯＮ',
    'ＢＡＮＣＵＡＮ',
    'ＣＩＮＮＡＭＯＮ',
    'ＢＵＦＦＡＬＯ ＨＯＲＮ',
    'ＢＥＡＤＳ',
    'ＰＩＧ ＳＵＥＤＥ',
    'ＲＡＴＴＡＮ',
    'ＴＥＲＲＡ ＣＯＴＴＡ',
    'ＲＵＢＢＩＴ ＦＵＲ',
    'ＢＵＦＦＡＬＯ ＬＥＡＴＨＥＲ',
    'ＡＲＴＩＣＨＯＫＥ',
    'ＣＯＣＯＮＵＴ ＦＩＢＥＲ',
    'ＰＩＮＥＣＯＲＮ',
    'ＦＥＲＮ ＬＥＡＦ',
    'ＧＡＬＶＡＮＡＩＺＥＤ',
    '牛革',
    '磁器',
    'シーグラス',
    'ＣＯＰＰＥＲ',
    '桜',
    'クロームメッキ',
    'アルパカ',
    'ＣＡＬＦ ＬＥＡＴＨＥＲ',
    'ＶＯＩＬ ＣＯＴＯＮ',
    'ＨＯＬＬＹ ＬＥＡＦ',
    'ＲＥＣＹＣＬＥ ＬＥＡＴＨＥＲ',
    'ＬＡＶＥＮＤＥＲ',
    'ＳＴＥＥＬ',
    'ＦＡＫＥ ＦＵＲ',
    'ＣＯＣＯＮＵＴ',
    'ＳＨＥＬＬ',
    'ＳＥＳＢＡＮＩＡ',
    'ＣＯＷ ＳＵＥＤＥ',
    'ＰＯＮＹ',
    'ムートン',
    'ＯＲＡＮＧＥ ＦＬＯＷＥＲ',
    'ＬＥＭＯＮ ＶＥＲＢＥＮＡ',
    'ＢＡＭＢＯＯ',
    'ＢＥＴＥＬ ＮＵＴ',
    'ＳＣＲＥＷ ＰＩＮＥ',
    'ＰＩＮＥ ＣＯＮＥ',
    'ＪＡＣＫＦＵＲＵＩＴ',
    'ＰＯＷＤＥＲ',
    'ＣＨＥＳＴＮＵＴ',
    'ＲＡＴＡＮ  ＷＩＣＫＥＲ',
    'ＦＡＫＥ ＬＥＡＴＨＥＲ',
    'ＯＲＡＮＧＥ ＦＬＯＷＥＲ',
    'ＬＥＭＯＮ ＶＥＲＢＥＮＡ',
    'ＣＡＭＥＬ ＢＯＮＥ',
    'ＣＥＬＵＬＯＩＤ',
    'ＰＩＮＥ ＣＯＲＮ',
    'ＡＬＵＭＩＮＵＭ',
    '紙',
    'ＨＥＭＰ',
    'ＡＢＴ',
    'ＧＩＮＧＥＲ ＳＰＩＣＥ',
    'ＧＡＲＤＥＮＩＡ',
    'ＰＥＡＲ',
    'ＣＲＡＹ ＰＯＴ',
    'ＢＵＴＴＥＲＦＬＹ ＬＥＡＦ',
    'ＪＡＣＫ ＦＲＵＩＴ ＬＥＡＦ',
    'ＢＥＴＥＬ ＮＵＴ',
    'ＳＣＲＥＷ ＰＩＮＥ',
    'ＰＩＮＥ ＣＯＮＥ',
    'ＪＡＣＫＦＵＲＵＩＴ',
    '亜鉛',
    'リストパール',
    'アルミ',
    'ウォッシュドウール',
    'アイアン',
    'ビーズ',
    'ＭＡＮＧＯ ＬＥＡＦ',
    'アンダリ',
    'ＬＥＡＴＨＥＲ',
    'ＷＩＬＬＯＷ',
    'シェル',
    'グラシア',
    'パール',
    'リネ',
    'ブロンズ',
    'ウッド',
    'アバカ',
    'リサイクル',
    'ウール',
    'アラン',
    'ストロー',
    'オイルレザー',
    'ゴム',
    'コットン',
    'レザー',
    'ラタン',
    'アメジスト',
    '木製',
    'ＦＵＲ',
    'ホーン',
    'horn',
    '革',
    'ラフィア',
    'ポリウレタン',
    'ワッフル',
    'クリスタル',
    'ウィロー',
    'オックス',
    'キャンバス',
    'ＲＡＦＦＩＡ',
    'モヘヤ',
    'ＭＯＨＡＩＲ'
]

def get_unique_list(series):
    new_series = [w.replace('\u3000', ' ') for w in series.tolist()]
    unique_series = list(set(new_series))
    return unique_series

unique_material_keywords =  get_unique_list(material_df['quality'].dropna()) + get_unique_list(material_df['product_name'])
unique_material_keywords
print("There are {} unique product names".format(len(unique_material_keywords)))
removing_indexes = []
for index, c in enumerate(unique_material_keywords.copy()):
    for s in query_keywords:
        if s in c:
            removing_indexes.append(index)

for index in sorted(list(set(removing_indexes)), reverse=True):
    del unique_material_keywords[index]            

 
print("{} of products still remain unassigned with material".format(len(unique_material_keywords)))

There are 14890 unique product names
6529 of products still remain unassigned with material


In [41]:
group_dict = {
    '革': ['牛革','合成皮革','オイルレザー','レザー','leather'],
    'ウール': ['ウォッシュドウール','ウール','wool','ツイストウール','フラッフィーウール'],
    'ウッド': ['木製','ウッド','wood'],
    '毛': ['カシミア','アルパカ','アンゴラ'],
    'コットン': ['綿','コットン'],
    'アルミニウム': ['aluminium','アルミニウム','aluminum','アルミ'],
    'ワックス': ['wax','ワックス'],
    'ホーン': ['horn','ホーン'],
    'ファー': ['fur','ファー'],
    '紙': ['paper','紙'],
    'シルバー': ['silver','シルバー'],
    'シルク': ['silk','シルク'],
    'ラフィア':['raffia','ラフィア'],
    'モヘヤ':['mohair','モヘヤ']
}


def get_groups(names,group_dict):
    groups = []
    for name in names:
        group_holder = []

        for key in group_dict:
            for query in group_dict[key]:
                if query in name:
                    group_holder.append(key)
                    break

        if group_holder:
            groups.append(",".join(group_holder))
        else:
            groups.append(name)
    return groups

query_keywords = sorted(query_keywords)
names = [ unicodedata.normalize('NFKC', name) for name in [ k.lower().replace(" ","") for k in query_keywords.copy()]]        
groups = get_groups(names,group_dict)

dictionary = { 'query_keywords': query_keywords, 'name': names, 'group': groups }
new_df = pd.DataFrame(dictionary)

### Export dictionaries as csv files

In [42]:
new_df.to_csv('processed_data/material.csv', header=True, mode='w')